<a href="https://colab.research.google.com/github/saivenkatreddy29/Implementation-of-Hands-on-Large-Language-models-book/blob/main/Jaylamar_book_chapter_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain==0.2.5 faiss-gpu==1.7.2 cohere==5.5.8 langchain-community==0.2.5 rank_bm25==0.2.2 sentence-transformers==3.0.1
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.78

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successf

In [2]:
import cohere

In [3]:
api_key = 'x8u7FBEcQq68MnLnTcXaFQNQIw9zHRjOKsiDL480'
co = cohere.Client(api_key)

In [4]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

# Split into a list of sentences
texts = text.split('.')
texts = [t.strip('\n') for t in texts]

# Embedding the text chunks

In [5]:
import numpy as np

response = co.embed(
    texts = texts,
    input_type = 'search_document'
).embeddings

embeds = np.array(response)
print(embeds.shape)

(15, 4096)


In [6]:
print(embeds)

[[ 0.21228027 -1.2519531   1.2949219  ...  1.5058594   0.88671875
  -0.79052734]
 [ 2.3847656  -0.5361328   0.4560547  ...  0.8027344   0.1430664
  -0.21972656]
 [ 1.6035156  -1.0830078   1.0039062  ...  0.32128906 -1.4453125
  -0.3137207 ]
 ...
 [ 2.0058594  -0.42797852  3.0253906  ...  0.84375    -1.9521484
  -0.23461914]
 [ 2.5039062  -1.4755859   1.8046875  ...  0.78808594 -0.14978027
  -1.0058594 ]
 [ 0.5527344  -2.6953125   1.1621094  ... -1.7802734  -1.7197266
  -2.1289062 ]]


# Building The Search Index

In [7]:
import faiss
dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

# Search the Index

In [8]:
import pandas as pd

def search(query, number_of_results = 3):

  # get the query embeddings
  query_embed = co.embed(texts = [query],
                         input_type = 'search_query').embeddings[0]

  distances,similar_items_ids = index.search(np.float32([query_embed]), number_of_results)

  texts_np = np.array(texts)
  results = pd.DataFrame(data = {'texts': texts_np[similar_items_ids[0]],
                                 'distance':distances[0]})
  # print the resutls
  print(f"query:{query} \n Nearest neighbors:")
  return results

In [9]:
query = "how precise was the science"
results = search(query)
results

query:how precise was the science 
 Nearest neighbors:


,texts,distance
0,It has also received praise from many astronom...,10757.379883
1,Caltech theoretical physicist and 2017 Nobel l...,11566.131836
2,Interstellar uses extensive practical and mini...,11922.833008


In [10]:
!pip install rank_bm25 -q

In [11]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
      tokenized_doc.append(token)
  return tokenized_doc

In [ ]:
from tqdm import tqdm

tokenized_corpus = []
for passage in tqdm(texts):
  tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

In [13]:
def keyword_search(query,top_k = 3, num_candidates = 15):
  print("Input question", query)

  ## BM25 search (lexical search)
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores,-num_candidates)[-num_candidates:]
  bm25_hits = [{'corpus_id': idx, 'score':bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key = lambda x:x['score'], reverse = True)

  print(f'Top 3 lexical search (BM25) hits')
  for hit in bm25_hits[0:top_k]:
    print("\t {:3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n"," ")))

In [14]:
keyword_search(query = 'how precise was the science')

Input question how precise was the science
Top 3 lexical search (BM25) hits
	 1.788605	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	 1.372650	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	 0.000000	It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine


 # Caveats of Dense Retrival

In [15]:
query = "what is the mass of moon"
results = search(query)
results

query:what is the mass of moon 
 Nearest neighbors:


,texts,distance
0,Cinematographer Hoyte van Hoytema shot it on 3...,13806.704102
1,It has also received praise from many astronom...,13908.627930
2,The film had a worldwide gross over $677 milli...,13971.905273


# Reranking Example

In [16]:
query = "how precise was the science"
results = co.rerank(query = query, documents = texts,top_n = 3, return_documents = True)
results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'), index=12, relevance_score=0.16981852),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'), index=10, relevance_score=0.07004896),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar'), index=4, relevance_score=0.0043994132)]

In [17]:
for idx,result in enumerate(results.results):
  print(idx, result.relevance_score, result.document.text)

0 0.16981852 It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics
1 0.07004896 The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014
2 0.0043994132 Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar


In [18]:
def keyword_and_reranking_search(query, top_k = 3, num_candidates = 10):
  print("Input question:", query)

  ## BM25 search(lexical search)
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
  bm25_hits = [{'corpus_id':idx, 'score':bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key = lambda x:x['score'], reverse = True)

  print(f'top 3 lexical search (BM25) hits')
  for hit in bm25_hits[0:top_k]:
    print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n"," ")))

  docs = [texts[hit['corpus_id']] for hit in bm25_hits]

  print(f'\nTop-3 hits by rank-API ({len(bm25_hits)} BM25 hits re-ranked)')
  results = co.rerank(query = query, documents = docs, top_n = top_k, return_documents = True)
  for hit in results.results:
    print("\t{:.3f}\t{}".format(hit.relevance_score,hit.document.text.replace("\n","")))

In [20]:
keyword_and_reranking_search(query= "how precise was the science")

Input question: how precise was the science
top 3 lexical search (BM25) hits
	1.789	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	1.373	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.000	Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects

Top-3 hits by rank-API (10 BM25 hits re-ranked)
	0.004	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.004	Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind
	0.003	Brothers Christopher and Jonat

# Retrival Augmented Generation

Grounded Generation with an LLM API

In [22]:
query = "income generated"

results = search(query)

docs_dict = [{"text": text} for text in results['texts']]
response = co.chat(
    message = query,
    documents = docs_dict
)
print(response.text)

query:income generated 
 Nearest neighbors:
The film Interstellar grossed over $677 million worldwide, with subsequent re-releases bringing the total to $773 million.


In [23]:
results

,texts,distance
0,The film had a worldwide gross over $677 milli...,14360.872070
1,Caltech theoretical physicist and 2017 Nobel l...,15292.041016
2,Interstellar uses extensive practical and mini...,15345.039062


In [24]:
response


NonStreamedChatResponse(text='The film Interstellar grossed over $677 million worldwide, with subsequent re-releases bringing the total to $773 million.', generation_id='010c12c9-6b8e-4c1d-823c-4a1ea83efd15', citations=[ChatCitation(start=22, end=57, text='grossed over $677 million worldwide', document_ids=['doc_0']), ChatCitation(start=64, end=122, text='subsequent re-releases bringing the total to $773 million.', document_ids=['doc_0'])], documents=[{'id': 'doc_0', 'text': 'The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'}], is_search_required=None, search_queries=None, search_results=None, finish_reason='COMPLETE', tool_calls=None, chat_history=[Message_User(message='income generated', tool_calls=None, role='USER'), Message_Chatbot(message='The film Interstellar grossed over $677 million worldwide, with subsequent re-releases bringing the total to $773 million.', tool_calls=None, role=

In [25]:
response.citations

[ChatCitation(start=22, end=57, text='grossed over $677 million worldwide', document_ids=['doc_0']),
 ChatCitation(start=64, end=122, text='subsequent re-releases bringing the total to $773 million.', document_ids=['doc_0'])]

Rag with local Models

In [26]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2024-11-25 01:30:57--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 3.165.160.59, 3.165.160.11, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1732757458&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjc1NzQ1OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4Z

In [31]:
!apt-get update
!apt-get install -y cuda

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,501 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,616 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [33]:
!ldconfig

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is

In [35]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path = "/content/Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers = 0,
    max_tokens = 500,
    n_ctx = 2048,
    seed = 42,
    verbose = False
)

RuntimeError: Failed to load shared library '/usr/local/lib/python3.10/dist-packages/llama_cpp/libllama.so': libcuda.so.1: cannot open shared object file: No such file or directory

In [38]:
!pip uninstall llama-cpp-python
!pip install llama-cpp-python-cpu --upgrade


Found existing installation: llama_cpp_python 0.2.78
Uninstalling llama_cpp_python-0.2.78:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/bin/convert-hf-to-gguf.py
    /usr/local/lib/python3.10/dist-packages/bin/llava-cli
    /usr/local/lib/python3.10/dist-packages/include/ggml-alloc.h
    /usr/local/lib/python3.10/dist-packages/include/ggml-backend.h
    /usr/local/lib/python3.10/dist-packages/include/ggml-cuda.h
    /usr/local/lib/python3.10/dist-packages/include/ggml.h
    /usr/local/lib/python3.10/dist-packages/include/llama.h
    /usr/local/lib/python3.10/dist-packages/lib/cmake/Llama/LlamaConfig.cmake
    /usr/local/lib/python3.10/dist-packages/lib/cmake/Llama/LlamaConfigVersion.cmake
    /usr/local/lib/python3.10/dist-packages/lib/libggml_shared.so
    /usr/local/lib/python3.10/dist-packages/lib/libllama.so
    /usr/local/lib/python3.10/dist-packages/lib/libllava.so
    /usr/local/lib/python3.10/dist-packages/lib/pkgconfig/llama.pc
    /usr/local/lib/python3.10/dist

In [39]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.2-cp310-cp310-linux_x86_64.whl size=3410010 sha256=9c289181b97a3cabe48209e34a2af1bc1a51a2334830557db16b03c4bbbf8a03
  Stored in directory: /root/.cache/pip/wheels/49/1c/f0/6c1ed6032d5827dea28df8a8df860d90ca1bcd7b3fa455f5e1
Successfully built llama-cpp-python


In [36]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="/content/Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=0,  # Disable GPU layers
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)


RuntimeError: Failed to load shared library '/usr/local/lib/python3.10/dist-packages/llama_cpp/libllama.so': libcuda.so.1: cannot open shared object file: No such file or directory